# Function Calling

Function calling is a powerful capability that allows LLMs to call external functions.

These functions can do anything, such as call APIs, perform computations, and more.

Magentic makes it really simple to use.

Filter out some Pydantic warnings

In [1]:
import warnings
warnings.filterwarnings("ignore")

Necessary imports

In [2]:
from dotenv import load_dotenv
from magentic import chatprompt, prompt, prompt_chain, SystemMessage, UserMessage, FunctionCall
from magentic.chat_model.litellm_chat_model import LitellmChatModel
from pydantic import BaseModel
import requests


load_dotenv()

True

# Define A Function To Get Realtime Weather

In [3]:

def get_weather(city_name: str) -> str:
    """
    Retrieves the weather information for a given city.

    Args:
        city_name (str): The name of the city for which to retrieve the weather information.

    Returns:
        str: The weather information for the specified city.
    """
    base_url = f"http://wttr.in/{city_name}?format=%C+%t"
    response = requests.get(base_url)
    return response.text

# Function Calling With Simple `@prompt`

In [4]:
@prompt(
    "Use the appropriate function to answer: {question}",
    functions=[get_weather],
    model=LitellmChatModel("gpt-3.5-turbo"),
)
def answer(question: str) -> FunctionCall[str]: ...


output = answer("What is the weather in Lausanne?")
print(output)
output()

FunctionCall(<function get_weather at 0x10e484680>, 'Lausanne')


'Light rain +10°C'

# Function Calling With `pompt_chain`

In [5]:
@prompt_chain(
    "Use the appropriate function to answer: {question}",
    functions=[get_weather],
    model=LitellmChatModel("gpt-3.5-turbo"),
)
def answer_chain(question: str) -> str: ...


answer_chain("What is the weather in Lausanne?")

'The weather in Lausanne is light rain with a temperature of +10°C.'

# Function Calling With `@chatprompt`

In [6]:
@chatprompt(
    SystemMessage("You are a weather beast. Use the proper function to answer."),
    UserMessage("{message}?"),
    model=LitellmChatModel("gpt-3.5-turbo"),
    functions=[get_weather],
)
def answer_chat(message: str) -> FunctionCall[str]: ...


output = answer_chat("What is the weather in Lausanne?")
output()

'Light rain +10°C'